<a href="https://colab.research.google.com/github/tvaditya/intro_ds_and_ml/blob/main/%5BNLP3%5DResumoComWikipedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##<font color="#fd79a8">Sumários gerados por extração <br><font color="#48dbfb">Scraping de artigo de Wikipedia para gerar resumos

In [1]:
import bs4 as bs
import urllib.request
import re
import nltk
nltk.download('punkt')
import sys
import csv 

#persian cuisine
scraped_data = urllib.request.urlopen('https://pt.wikipedia.org/wiki/Cart%C3%A3o_private_label')
article = scraped_data.read()

parsed_article = bs.BeautifulSoup(article,'lxml')

paragraphs = parsed_article.find_all('p')

article_text = ""

for p in paragraphs:
    article_text += p.text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
# Removing Square Brackets and Extra Spaces
article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)
#any whitespace character \s+
article_text = re.sub(r'\s+', ' ', article_text)

In [3]:
# Removing special characters and digits
formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )
#any whitespace character \s+
formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)

##<font color="#fd79a8">Converter paragrafos em sentenças

In [4]:
sentence_list = nltk.sent_tokenize(article_text)

In [5]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


###<font color="#fd79a8"> Calcular a frequencia das palavras. <br>Tokenize as sentenças <br>Se não for uma stopword e não estiver incluido nas lista de palavras , adicione e conte

In [6]:
# stopwords = nltk.corpus.stopwords.words('english')

word_frequencies = {}
for word in nltk.word_tokenize(formatted_article_text):
    if word not in stopwords:
        if word not in word_frequencies.keys():
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1


In [7]:
word_frequencies

{'A': 2,
 'ABECS': 1,
 'American': 1,
 'Associa': 1,
 'Aumento': 1,
 'Brasil': 3,
 'Brasileira': 1,
 'CPF': 1,
 'Cart': 2,
 'Co': 1,
 'Com': 1,
 'Cr': 2,
 'De': 1,
 'Desburocratiza': 1,
 'Dinners': 1,
 'E': 1,
 'Empresas': 1,
 'Esta': 1,
 'Express': 1,
 'Fideliza': 1,
 'Ganha': 1,
 'Label': 2,
 'Mappin': 1,
 'Mastercard': 1,
 'Mesbla': 1,
 'N': 1,
 'No': 1,
 'O': 2,
 'Obter': 1,
 'P': 1,
 'Para': 2,
 'Parcerias': 1,
 'Por': 1,
 'Private': 2,
 'Servi': 1,
 'Seu': 1,
 'Tamb': 1,
 'Visa': 1,
 'aceita': 2,
 'acordo': 1,
 'adquirentes': 1,
 'afiliadas': 1,
 'afinal': 1,
 'al': 1,
 'alvo': 1,
 'an': 1,
 'apenas': 2,
 'apresentar': 1,
 'ar': 1,
 'assertividade': 1,
 'aumenta': 1,
 'autoriza': 1,
 'aux': 1,
 'banc': 1,
 'banco': 1,
 'bandeira': 2,
 'bandeirado': 1,
 'bandeiras': 2,
 'basta': 1,
 'blico': 1,
 'branded': 1,
 'cada': 1,
 'cadastro': 1,
 'cadeia': 1,
 'caracter': 1,
 'cart': 14,
 'certamente': 1,
 'certeiro': 1,
 'chamado': 1,
 'cia': 1,
 'cias': 1,
 'cio': 1,
 'cliente': 6,
 'cli

###<font color="#48dbfb">Calcular o weighted frequency de ocorrencia

In [8]:
maximum_frequncy = max(word_frequencies.values())

for word in word_frequencies.keys():
    word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

###<font color="#48dbfb">Substituir as palavras pelo weighted frequency nas sentenças

In [9]:
sentence_scores = {}
for sent in sentence_list:
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(' ')) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]

In [10]:
sentence_scores

{'A efetivação de qualquer negócio será entre lojista e cliente.': 0.5714285714285714,
 'Aumento certeiro em número de clientes e volume de vendas.': 0.2857142857142857,
 'Com o cartão private label ele poderá parcelar as compras, contratar financiamentos com juros e prazos diferenciados e sem a autorização do sistema financeiro.': 1.6428571428571426,
 'Desburocratização A obtenção do cartão private label é rápida e totalmente desburocratizada.': 0.9999999999999999,
 'E mais interessante em se trabalhar com esta modalidade: não é necessário que o cliente tenha conta em banco, o que significa operar num universo de 40% da população.': 0.9999999999999998,
 'Ganha seu cliente, ganha você.': 0.5714285714285714,
 'No momento de realizar este controle, é importante ter o auxílio de um sistema de gestão, afinal a mensuração de resultados se dará com muito mais eficácia e assertividade.': 0.857142857142857,
 'Não há a necessidade de grandes análises de cadastro.': 0.21428571428571427,
 'Obter 

####<font color="#fd79a8">Heap queue Pegar as 5 maiores frases pelo score gerado<br><font color="#0abde3">heapq.nlargest(n, iterable, key=None)

In [11]:
import heapq
summary_sentences = heapq.nlargest(5, sentence_scores, key=sentence_scores.get)

summary = ' '.join(summary_sentences)
summary

'Com o cartão private label ele poderá parcelar as compras, contratar financiamentos com juros e prazos diferenciados e sem a autorização do sistema financeiro. Parcerias O cartão private label também permite estabelecer e reforçar parcerias com outros estabelecimentos. É diferente dos cartões de crédito de uso universal, pois não tem uma bandeira de aceitação universal em todo o comércio, tais como as bandeiras Mastercard,American Express,Visa e Dinners. Desburocratização A obtenção do cartão private label é rápida e totalmente desburocratizada. E mais interessante em se trabalhar com esta modalidade: não é necessário que o cliente tenha conta em banco, o que significa operar num universo de 40% da população.'

In [ ]:
# Entrar em outro wikipedia (artigo)
# Aumentar o numero de frases